In [1]:
import uqcreator as uqc

In [2]:
settings = uqc.Settings()

settings.mesh['file'] = 'naca_profile.su2'
settings.mesh['bc'] = [["farfield","dirichlet"], ["airfoil", "noslip"]]

settings.problem['uncertainties'] = [["Legendre"],[0.5]]
settings.problem['cfl'] = 0.9
settings.problem['tEnd'] = 1.0

settings.moment_system['moments'] = [["totalDegree"],[8]]
settings.moment_system['quadPoints'] = [["tensorizedCCGrid"],[4]]

In [3]:
problem = uqc.Euler()
closure = uqc.StochasticGalerkin()

TypeError: Can't instantiate abstract class Euler with abstract method ComputeDt